In [1]:
import numpy as np
from numpy.linalg import norm
from scipy.linalg import kron
from scipy.stats import unitary_group
from scipy.sparse.linalg import svds, eigsh
import random
from scipy.linalg import expm, sinm, cosm
np.set_printoptions(precision=3)
N = 10 # number of spins
d = 2 # dim

In [2]:
sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
sz = np.array([[1,0],[0,-1]])
sxsx = kron(sx, sx)
szsz = kron(sz, sz)
sysy = kron(sy, sy)
h = sxsx + sysy + szsz

In [3]:
U = unitary_group.rvs(2)
UU = kron(U, U)
print((UU.conj().T@szsz@UU))

[[ 0.479+0.j    -0.109+0.488j -0.109+0.488j -0.472-0.222j]
 [-0.109-0.488j -0.479+0.j     0.521+0.j     0.109-0.488j]
 [-0.109-0.488j  0.521+0.j    -0.479+0.j     0.109-0.488j]
 [-0.472+0.222j  0.109+0.488j  0.109+0.488j  0.479+0.j   ]]


In [4]:
H = 0
for i in range(N-1):
    H += np.kron(np.eye(d**i), np.kron(h, np.eye(d**(N-2-i))))
w, v = eigsh(H, k=1)
print("E/N=", w[0]/N)

E/N= -1.7032140829131592


In [5]:
def random_pauli():
    return random.choice([np.eye(d), sx, sy, sz])
W = random_pauli()
for i in range(N-1):
    W = kron(W, random_pauli())

In [6]:
def make_p(l):
    v = np.array([[1],[0]]) if l[0]==0 else np.array([[0],[1]]) 
    for x in l[1:]:
        qbit = np.array([[1],[0]]) if x==0 else np.array([[0],[1]]) 
        v = kron(v, qbit)
    return v

onep = 0
for i in range(N):
    r = [1 if j==i else 0 for j in range(N)]
    onep = onep + make_p(r)@make_p(r).conj().T

In [7]:
dtheta = 0.01
def random_pauli():
    return random.choice([np.eye(d), sx, sy, sz])
W = random_pauli()
for i in range(N-1):
    W = kron(W, random_pauli())
np.linalg.norm(W - W.conj().T)
F = W@onep@W.conj().T
np.linalg.norm(F - F.conj().T)

0.0

In [8]:
np.linalg.norm(F)

3.1622776601683795

In [10]:
def eval_energy(W):
    zeroket = make_p([0]*N)
    psiket = W@zeroket
    E = psiket.conj().T@H@psiket
    return E[0][0]/N

In [13]:
dtheta = 100
def random_pauli():
    return random.choice([np.eye(d), sx, sy, sz])
W = random_pauli()
for i in range(N-1):
    W = kron(W, random_pauli())

for i in range(10):
    print(eval_energy(W))
    F = W@onep@W.conj().T
    A = H@F - F@H
    Ud = expm(dtheta*A)
    W = Ud@W

(-0.1+0j)
(-0.09999999999999716+0j)
(-0.09999999999999148+0j)
(-0.0999999999999858+0j)
(-0.09999999999997443+0j)
(-0.09999999999996306+0j)
(-0.09999999999995168+0j)
(-0.09999999999994032+0j)
(-0.09999999999992895+0j)
(-0.09999999999991759+0j)


In [19]:
zeroket = make_p([0]*N)
psiket = W@zeroket
E = psiket.conj().T@H@psiket
psiket[0]

array([0.+0.j])